In [137]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.datasets import load_digits
from sklearn import datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F



In [138]:
class Digits(Dataset):
    """Scikit-Learn Digits dataset."""

    def __init__(self, mode="train", transforms=None):
        digits = load_digits()
        if mode == "train":
            self.data = digits.data[:1000].astype(np.float32)
        elif mode == "val":
            self.data = digits.data[1000:1350].astype(np.float32)
        else:
            self.data = digits.data[1350:].astype(np.float32)

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transforms:
            sample = self.transforms(sample)
        return sample

In [139]:
data = Digits()
data[0].shape
data

In [140]:
class RoundStraightThrough(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return torch.round(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clone()


In [141]:
def log_min_exp(a, b):
    # Numerically stable log(exp(a) - exp(b)).
    max_ab = torch.maximum(a, b)
    min_ab = torch.minimum(a, b)
    return max_ab + torch.log1p(-torch.exp(min_ab - max_ab))

In [142]:
def log_integer_probability(x, mean, logscale):
    logscale = torch.clamp(logscale, -5, 5)
    scale = torch.exp(logscale)
    logp = log_min_exp(
        F.logsigmoid(((x + 0.5) - mean) / scale),
        F.logsigmoid(((x - 0.5) - mean) / scale)
    )
    return logp

In [143]:
D = 64   # input dimension
M = 256  # the number of neurons in scale (s) and translation (t) nets

lr = 1e-3 # learning rate
num_epochs = 100 # max. number of epochs
max_patience = 20 # an early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped
nett = lambda: nn.Sequential(nn.Linear(D // 2, M), nn.LeakyReLU(),
                                     nn.Linear(M, M), nn.LeakyReLU(),
                                     nn.Linear(M, D // 2))
netts = [nett]

In [ ]:
class IDF(nn.Module):
    def __init__(self, netts, num_flows, D=2):
        super().__init__()
        if len(netts) == 1:
            self.t = nn.ModuleList([netts[0]() for _ in range(num_flows)])
            self.idf_git = 1

        elif len(netts) == 4:
            self.t_a = nn.ModuleList([netts[0]() for _ in range(num_flows)])
            self.t_b = nn.ModuleList([netts[1]() for _ in range(num_flows)])
            self.t_c = nn.ModuleList([netts[2]() for _ in range(num_flows)])
            self.t_d = nn.ModuleList([netts[3]() for _ in range(num_flows)])
            self.idf_git = 1

        else:
            raise ValueError(f"The transformation net need to be either 1 or 4. The provided net contains {len(netts)} layers.")

        self.num_flows = num_flows
        self.round = RoundStraightThrough.apply

        self.mean = nn.Parameter(torch.zeros(1, D))
        self.logscale = nn.Parameter(torch.ones(1, D))

        self.D = D

    def coupling(self, x, index, forward=True):
        if self.idf_git == 1:
            (xa, xb) = torch.chunk(x, 2, 1)

            if forward:
                yb = xb + self.round(self.t[index](xa))
            else:
                yb = xb - self.round(self.t[index](xa))
            return torch.cat((xa, yb), 1)
        elif self.idf_git == 4:
            (xa, xb, xc, xd) = torch.chunk(x, 4, 1)
            if forward:
                ya = xa + self.round(self.t_a[index](torch.cat((xb, xc, xd), 1)))
                yb = xb + self.round(self.t_b[index](torch.cat((ya, xc, xd), 1)))
                yc = xc + self.round(self.t_c[index](torch.cat((ya, yb, xd), 1)))
                yd = xd + self.round(self.t_d[index](torch.cat((ya, yb, yc), 1)))
            else:
                yd = xd - self.round(self.t_d[index](torch.cat((xa, xb, xc), 1)))
                yc = xc - self.round(self.t_c[index](torch.cat((xa, xb, yd), 1)))
                yb = xb - self.round(self.t_b[index](torch.cat((xa, yc, yd), 1)))
                ya = xa - self.round(self.t_a[index](torch.cat((yb, yc, yd), 1)))

            return torch.cat((ya, yb, yc, yd), 1)

    def permute(self, x):
        return x.flip(1)

    def f(self, x):
        z = x
        for i in range(self.num_flows):
            z = self.coupling(z, i, forward=True)
            z = self.permute(z)
        return z

    def f_inv(self, z):
        x = z
        for i in reversed(range(self.num_flows)):
            x = self.permute(x)
            x = self.coupling(x, i, forward=False)

        return x

    def log_prior(self, x):
        x = torch.clamp(x, -20, 20)
        log_p = log_integer_probability(x, self.mean, self.logscale)
        return log_p.sum(1)

    def forward(self, x, reduction="avg"):
        z = self.f(x)
        if reduction == 'sum':
            return -self.log_prior(z).sum()
        else:
            return -self.log_prior(z).mean()

    def sample(self, batch_size, int_max=100):
        z = self.prior_sample(batch_size=batch_size, D=self.D)
        x = self.f_inv(z)
        return x  # shape: (batch_size, D)

    def prior_sample(self, batch_size, D=2):
        y = torch.rand(batch_size, self.D)
        x = torch.exp(self.logscale) * torch.log(y / (1. - y)) + self.mean
        return torch.round(x)

In [145]:
IDF(netts, 1)

IDF(
  (t): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=32, out_features=256, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=32, bias=True)
    )
  )
)

In [146]:
from torchinfo import summary

num_flows = 8

idf_git = 4

if idf_git == 1:
    nett = lambda: nn.Sequential(
        nn.Linear(D // 2, M),
        nn.LeakyReLU(),
        nn.Linear(M, M),
        nn.LeakyReLU(),
        nn.Linear(M, D // 2),
    )
    netts = [nett]

elif idf_git == 4:
    nett_a = lambda: nn.Sequential(
        nn.Linear(3 * (D // 4), M),
        nn.LeakyReLU(),
        nn.Linear(M, M),
        nn.LeakyReLU(),
        nn.Linear(M, D // 4),
    )

    nett_b = lambda: nn.Sequential(
        nn.Linear(3 * (D // 4), M),
        nn.LeakyReLU(),
        nn.Linear(M, M),
        nn.LeakyReLU(),
        nn.Linear(M, D // 4),
    )

    nett_c = lambda: nn.Sequential(
        nn.Linear(3 * (D // 4), M),
        nn.LeakyReLU(),
        nn.Linear(M, M),
        nn.LeakyReLU(),
        nn.Linear(M, D // 4),
    )

    nett_d = lambda: nn.Sequential(
        nn.Linear(3 * (D // 4), M),
        nn.LeakyReLU(),
        nn.Linear(M, M),
        nn.LeakyReLU(),
        nn.Linear(M, D // 4),
    )

    netts = [nett_a, nett_b, nett_c, nett_d]

model = IDF(netts, num_flows, D=D)

model.idf_git = 4  # ensure 4-way coupling path is used

summary(
    model=model,
    input_size=(1, D),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"],
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
IDF (IDF)                                [1, 64]              --                   128                  True
├─ModuleList (t_a)                       --                   --                   (recursive)          True
│    └─Sequential (0)                    [1, 48]              [1, 16]              --                   True
│    │    └─Linear (0)                   [1, 48]              [1, 256]             12,544               True
│    │    └─LeakyReLU (1)                [1, 256]             [1, 256]             --                   --
│    │    └─Linear (2)                   [1, 256]             [1, 256]             65,792               True
│    │    └─LeakyReLU (3)                [1, 256]             [1, 256]             --                   --
│    │    └─Linear (4)                   [1, 256]             [1, 16]              4,112                True
├─ModuleList (t_b)

In [147]:
def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + ".model")

    model_best.eval()
    loss = 0.0
    N = 0.0
    for indx_batch, test_batch in enumerate(test_loader):
        loss_t = model_best.forward(test_batch, reduction="sum")
        loss = loss + loss_t.item()
        N = N + test_batch.shape[0]
    loss = loss / N

    if epoch is None:
        print(f"FINAL LOSS: nll={loss}")
    else:
        print(f"Epoch: {epoch}, val nll={loss}")

    return loss

In [148]:
from torch.utils.data import DataLoader

train_data = Digits(mode="train")
val_data = Digits(mode="val")
test_data = Digits(mode="test")

training_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [149]:
from tqdm.auto import tqdm
def training(name, model, device="cpu"):
    nll_val = []
    best_nll = 1000.0
    patience = 0
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in tqdm(range(num_epochs)):
        for x in tqdm(training_loader):
            loss = model.forward(x)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
        loss_val = evaluation(val_loader, model_best=model, epoch=epoch)
        nll_val.append(loss_val)  # save for plotting
        print(loss_val)
    nll_val = np.asarray(nll_val)

    return nll_val

In [150]:
training(name="IDF", model=model)

  1%|          | 1/100 [00:01<03:04,  1.86s/it]

Epoch: 0, val nll=428.6674776785714
428.6674776785714


  2%|▏         | 2/100 [00:03<02:48,  1.72s/it]

Epoch: 1, val nll=362.94489118303574
362.94489118303574


  3%|▎         | 3/100 [00:05<02:51,  1.77s/it]

...

 99%|█████████▉| 99/100 [03:51<00:02,  2.36s/it]

Epoch: 98, val nll=0.07063724790300642
0.07063724790300642


100%|██████████| 100/100 [03:53<00:00,  2.34s/it]

Epoch: 99, val nll=0.06661463499069215
0.06661463499069215


array([4.28667478e+02, 3.62944891e+02, 3.27046875e+02, 3.07277467e+02,
       2.96243622e+02, 2.89971431e+02, 2.86313323e+02, 2.84100432e+02,
       2.82688996e+02, 2.81719450e+02, 2.80988557e+02, 2.80380829e+02,
       2.79830354e+02, 2.79299774e+02, 2.78767972e+02, 2.78223175e+02,
       2.77658722e+02, 2.77070831e+02, 2.76457280e+02, 2.75816406e+02,
       2.75147068e+02, 2.74448119e+02, 2.73718538e+02, 2.72957277e+02,
       2.72163055e+02, 2.71334503e+02, 2.70470246e+02, 2.69568818e+02,
       2.68628389e+02, 2.67647183e+02, 2.66623076e+02, 2.65553989e+02,
       2.64437376e+02, 2.63270597e+02, 2.62050657e+02, 2.60774383e+02,
       2.59438018e+02, 2.58037627e+02, 2.56568753e+02, 2.55026367e+02,
       2.53404814e+02, 2.51697769e+02, 2.49898073e+02, 2.47997420e+02,
       2.45986539e+02, 2.43854492e+02, 2.41588728e+02, 2.39174527e+02,
       2.36594463e+02, 2.33827847e+02, 2.30849820e+02, 2.27629992e+02,
       2.24130963e+02, 2.20305910e+02, 2.16095109e+02, 2.11421109e+02,
      

In [152]:

# Generate and visualize samples
model.eval()
with torch.no_grad():
    raw_samples = model.sample(batch_size=16).detach().cpu()
if raw_samples.dim() == 3:
    samples = raw_samples[:, 0, :]
else:
    samples = raw_samples
samples_np = samples.numpy()
fig, axes = plt.subplots(4, 4, figsize=(6, 6))
for ax, img in zip(axes.flat, samples_np):
    ax.imshow(img.reshape(8, 8), cmap="gray")
    ax.axis("off")
plt.tight_layout()
plt.show()


RuntimeError: shape '[16, 2, 64]' is invalid for input of size 1024